# Introduction/Business Problem:

### Location services have created new capabilities and existing businesses are using them more and more each day. Start-ups leveraging new technology and innovative tools have found themselves in a more capable role of being profitable and competitive by using location data to identify consumer patterns, tastes and trends. Location data is available from a number of social media platforms to include but are not limited to Google maps and Foursquare. 

### The owner of Acme Custom Bookmarks is considering moving her company to a larger market. Acme Custom Bookmarks is a novelty store that sells custom made book marks, book lights and other accessories that improve the customer’s reading experience. She has just hired a Data Scientist, to analyze the data and answer the question: “Given two major cities, which neighborhood, provides the optimal location for a niche book accessory company?”

### The start-up is a niche, so it does not face much competition. However, as a niche company, its customer base tends to be very small. If this company chooses the wrong location, it very well may close its doors shortly after opening them.



# Description of the Data

### To answer the business question. The data scientist will collect location data from Foursquare and apply data science techniques and tools. In particular, the Data Scientist will use data visualization to answer the Business Problem. The data collected will involve comparison of the two prominent locations to determine which is the best location to recommend to the owner.

### The following imports are for the express purpose of handling the data.

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


### The following is needed to access Foursquare's location data. We will be exploring two major cities/neighborhoods. Downtown Toronto and Manhattan New York.

In [3]:
CLIENT_ID = 'PAXP4SMGDP1XHXKMFSMYYVHJQI15N0I4NTNQSW5CYFWSGF0I' # your Foursquare ID
CLIENT_SECRET = '4RTI54VWBMSQPSKYL5OGNYQVDKLYDYXCLOQTTQBOABRBB4XI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PAXP4SMGDP1XHXKMFSMYYVHJQI15N0I4NTNQSW5CYFWSGF0I
CLIENT_SECRET:4RTI54VWBMSQPSKYL5OGNYQVDKLYDYXCLOQTTQBOABRBB4XI


# Methodology

### Acme sells a variety of book accesories including book lights, book stands and bookmarks. As a complimentary good to books in brick and mortar stores, the best place for Acme to place its store is near the highest concentration of books. To discover this location, the Data scientist will graph the locations of the book related area in each neighborhood.

### the search query consist of the term 'book' within a ten-thousand meter radius.

In [4]:
search_query = 'book'
radius = 10000
print(search_query + ' .... OK!')

book .... OK!


# Results

# First we will look at Downtown Toronto.

In [5]:
address = 'Downtown Toronto, CA'

geolocator = Nominatim()
location = geolocator.geocode(address)
neighborhood_latitude = location.latitude
neighborhood_longitude = location.longitude
print(neighborhood_latitude, neighborhood_longitude)

/anaconda3/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


43.655115 -79.380219


In [6]:

LIMIT =100

In [7]:
 # create URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, search_query, radius, LIMIT)

url # display URL

'https://api.foursquare.com/v2/venues/search?client_id=PAXP4SMGDP1XHXKMFSMYYVHJQI15N0I4NTNQSW5CYFWSGF0I&client_secret=4RTI54VWBMSQPSKYL5OGNYQVDKLYDYXCLOQTTQBOABRBB4XI&ll=43.655115,-79.380219&v=20180605&query=book&radius=10000&limit=100'

In [8]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5bc8ca6e1ed219428a2a0c78'},
 'response': {'venues': [{'id': '4bfda30a8992a593156cacb0',
    'name': 'Thomas Fisher Rare Book Library',
    'location': {'address': '120 St. George St',
     'crossStreet': 'University of Toronto',
     'lat': 43.664249170185364,
     'lng': -79.39900070428848,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.664249170185364,
       'lng': -79.39900070428848}],
     'distance': 1822,
     'postalCode': 'M5S 1A5',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['120 St. George St (University of Toronto)',
      'Toronto ON M5S 1A5',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1a7941735',
      'name': 'College Library',
      'pluralName': 'College Libraries',
      'shortName': 'Library',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/library_',
       'suffix': '.png'},
      'primary': True}],
    

In [10]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
df = json_normalize(venues)
df.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1a7941735', 'name': 'C...",False,4bfda30a8992a593156cacb0,120 St. George St,CA,Toronto,Canada,University of Toronto,1822,"[120 St. George St (University of Toronto), To...","[{'label': 'display', 'lat': 43.66424917018536...",43.664249,-79.399001,NaN,M5S 1A5,ON,Thomas Fisher Rare Book Library,v-1539885678,NaN
1,"[{'id': '50328a8e91d4c4b30a586d6c', 'name': 'N...",False,4d3869ab3d958cfa4eb4b271,350 Berkeley St,CA,Toronto,Canada,NaN,1199,"[350 Berkeley St, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.66188195154671...",43.661882,-79.368625,NaN,NaN,ON,The Children's Book Bank,v-1539885678,NaN
2,"[{'id': '4bf58dd8d48988d1b1941735', 'name': 'C...",False,4ba7ac52f964a52079a839e3,214 College Street,CA,Toronto,Canada,University of Toronto,1409,"[214 College Street (University of Toronto), T...","[{'label': 'display', 'lat': 43.65888825647875...",43.658888,-79.396929,NaN,M5T 3A1,ON,U of T Book Store,v-1539885678,NaN
3,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",False,4b7846f5f964a52079c12ee3,348 Danforth Avenue,CA,Toronto,Canada,near Chester,3325,"[348 Danforth Avenue (near Chester), Toronto O...","[{'label': 'display', 'lat': 43.67741319707514...",43.677413,-79.352734,NaN,NaN,ON,Book City,v-1539885678,NaN
4,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",False,4b2a6eb8f964a52012a924e3,220 Yonge St,CA,Toronto,Canada,NaN,182,"[220 Yonge St, Toronto ON M5B 2H1, Canada]","[{'label': 'display', 'lat': 43.65351471121164...",43.653515,-79.380696,Downtown Yonge,M5B 2H1,ON,Indigo,v-1539885678,NaN


In [11]:

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']
df_filtered = df.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_filtered['categories'] = df_filtered.apply(get_category_type, axis=1)

In [12]:
df_filtered.name


0                       Thomas Fisher Rare Book Library
1                              The Children's Book Bank
2                                     U of T Book Store
3                                             Book City
4                                                Indigo
5                            Book Your Road Test Online
6                               George Brown Book Store
7                                             Book Camp
8                                       For Book Of Men
9                                       Book Beautician
10                       Lakehead University Book Store
11                                     Eliot's Bookshop
12                                   Recycled Book Shop
13                                Book City (The Beach)
14                             Chan Sheung Kee Book Co.
15                                 Bob Miller Book Room
16                                 Anglican Book Centre
17                                        Fab Bo

In [13]:
df_filtered.rename(columns={'location.address':'address','location.cc':'cc','location.city':'city','location.country':'country','location.crossStreet':'crossStreet','location.distance':'distance','location.formattedAddress': 'formattedAddress','location.labeledLatLngs':'labeledLatLngs','location.lat':'lat','location.lng':'lng','location.postalCode':'postalCode','location.state':'state'})

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,location.neighborhood,postalCode,state,id
0,Thomas Fisher Rare Book Library,College Library,120 St. George St,CA,Toronto,Canada,University of Toronto,1822,"[120 St. George St (University of Toronto), To...","[{'label': 'display', 'lat': 43.66424917018536...",43.664249,-79.399001,NaN,M5S 1A5,ON,4bfda30a8992a593156cacb0
1,The Children's Book Bank,Non-Profit,350 Berkeley St,CA,Toronto,Canada,NaN,1199,"[350 Berkeley St, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.66188195154671...",43.661882,-79.368625,NaN,NaN,ON,4d3869ab3d958cfa4eb4b271
2,U of T Book Store,College Bookstore,214 College Street,CA,Toronto,Canada,University of Toronto,1409,"[214 College Street (University of Toronto), T...","[{'label': 'display', 'lat': 43.65888825647875...",43.658888,-79.396929,NaN,M5T 3A1,ON,4ba7ac52f964a52079a839e3
3,Book City,Bookstore,348 Danforth Avenue,CA,Toronto,Canada,near Chester,3325,"[348 Danforth Avenue (near Chester), Toronto O...","[{'label': 'display', 'lat': 43.67741319707514...",43.677413,-79.352734,NaN,NaN,ON,4b7846f5f964a52079c12ee3
4,Indigo,Bookstore,220 Yonge St,CA,Toronto,Canada,NaN,182,"[220 Yonge St, Toronto ON M5B 2H1, Canada]","[{'label': 'display', 'lat': 43.65351471121164...",43.653515,-79.380696,Downtown Yonge,M5B 2H1,ON,4b2a6eb8f964a52012a924e3
5,Book Your Road Test Online,Driving School,"Suite 2201, 250 younge street,",CA,Toronto,Canada,NaN,348,"[Suite 2201, 250 younge street,, Toronto ON M5...","[{'label': 'display', 'lat': 43.65359012929569...",43.653590,-79.383999,NaN,M5B 2L7,ON,57a98629498e94c824f44cf3
6,George Brown Book Store,Bookstore,200 King St. E,CA,Toronto,Canada,NaN,968,"[200 King St. E, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65098368560166...",43.650984,-79.369630,NaN,NaN,ON,50805023e4b0fedebc76d278
7,Book Camp,Arts & Entertainment,NaN,CA,Toronto,Canada,NaN,432,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.658488, 'lng':...",43.658488,-79.377560,NaN,NaN,ON,4e4fdc812271a1bdc3e3ff67
8,For Book Of Men,General Entertainment,NaN,CA,NaN,Canada,NaN,320,[Canada],"[{'label': 'display', 'lat': 43.652551078298, ...",43.652551,-79.378401,NaN,NaN,NaN,4f90a56ae4b0655201cc8496
9,Book Beautician,Health & Beauty Service,"913 Bay St, Toronto, ON, M5S 3L6",CA,Toronto,Canada,NaN,504,"[913 Bay St, Toronto, ON, M5S 3L6, Toronto ON ...","[{'label': 'display', 'lat': 43.65383853190168...",43.653839,-79.386231,NaN,M5S 3L6,ON,5acb038fb3d8e26a9ee6a1c4


## Now to display the map.

In [14]:
venues_map = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=13) # generate map

# add a red circle marker to represent the Center of Downtown Toronto
folium.features.CircleMarker(
    [neighborhood_latitude, neighborhood_longitude],
    radius=10,
    color='red',
    popup='Center',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the book related venues as blue circle markers
for lat, lng, label in zip(df_filtered['location.lat'], df_filtered['location.lng'], df_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

# Next we will look at Downtown Miami!

In [15]:
CLIENT_ID = 'PAXP4SMGDP1XHXKMFSMYYVHJQI15N0I4NTNQSW5CYFWSGF0I' # your Foursquare ID
CLIENT_SECRET = '4RTI54VWBMSQPSKYL5OGNYQVDKLYDYXCLOQTTQBOABRBB4XI' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PAXP4SMGDP1XHXKMFSMYYVHJQI15N0I4NTNQSW5CYFWSGF0I
CLIENT_SECRET:4RTI54VWBMSQPSKYL5OGNYQVDKLYDYXCLOQTTQBOABRBB4XI


In [53]:
address = 'Downtown Miami, FL'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

/anaconda3/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


25.82088515 -80.3340115143607


In [54]:
search_query = 'Book'
radius = 10000
print(search_query + ' .... OK!')

Book .... OK!


In [55]:
url1 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url1

'https://api.foursquare.com/v2/venues/search?client_id=PAXP4SMGDP1XHXKMFSMYYVHJQI15N0I4NTNQSW5CYFWSGF0I&client_secret=4RTI54VWBMSQPSKYL5OGNYQVDKLYDYXCLOQTTQBOABRBB4XI&ll=25.82088515,-80.3340115143607&v=20180604&query=Book&radius=10000&limit=100'

In [56]:
results1 = requests.get(url1).json()
results1

{'meta': {'code': 200, 'requestId': '5bc8cee6f594df1de608dca4'},
 'response': {'venues': [{'id': '4ad0d474f964a52037da20e3',
    'name': 'Books & Books',
    'location': {'address': '265 Aragon Ave',
     'lat': 25.75060375407932,
     'lng': -80.25994355195199,
     'labeledLatLngs': [{'label': 'display',
       'lat': 25.75060375407932,
       'lng': -80.25994355195199}],
     'distance': 10785,
     'postalCode': '33134',
     'cc': 'US',
     'city': 'Coral Gables',
     'state': 'FL',
     'country': 'United States',
     'formattedAddress': ['265 Aragon Ave',
      'Coral Gables, FL 33134',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d114951735',
      'name': 'Bookstore',
      'pluralName': 'Bookstores',
      'shortName': 'Bookstore',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/bookstore_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1539886822',
    'hasPerk': False},
   {'id': '58a5eba1bf1a6d0b9fedd

In [57]:
# assign relevant part of JSON to venues
venues1 = results1['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues1)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",False,4ad0d474f964a52037da20e3,265 Aragon Ave,US,Coral Gables,United States,NaN,10785,"[265 Aragon Ave, Coral Gables, FL 33134, Unite...","[{'label': 'display', 'lat': 25.75060375407932...",25.750604,-80.259944,33134,FL,Books & Books,v-1539886822,NaN
1,"[{'id': '503287a291d4c4b30a586d65', 'name': 'F...",False,58a5eba1bf1a6d0b9fedd1e3,8333 NW,US,Doral,United States,"53rd Street, Suite 450",199,"[8333 NW (53rd Street, Suite 450), Doral, FL 3...","[{'label': 'display', 'lat': 25.82177731876308...",25.821777,-80.332280,33166,FL,Booksmart Tax,v-1539886822,NaN
2,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4f4caf3de4b0b2a2fce7a993,NaN,US,Miami,United States,NaN,6467,"[Miami, FL 33126, United States]","[{'label': 'display', 'lat': 25.79749541746666...",25.797495,-80.274939,33126,FL,Book Link Cafe,v-1539886822,NaN
3,"[{'id': '4bf58dd8d48988d1f6931735', 'name': 'G...",False,57149a2c498ece023556c1d5,8100 NW 53rd St Apt 275,US,Doral,United States,NaN,429,"[8100 NW 53rd St Apt 275, Doral, FL 33166, Uni...","[{'label': 'display', 'lat': 25.82170730116426...",25.821707,-80.329826,33166,FL,Rooms2night Booking Engine,v-1539886822,NaN
4,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",False,517982a2e4b0545603eb185b,NaN,US,Miami Springs,United States,NaN,5336,"[Miami Springs, FL, United States]","[{'label': 'display', 'lat': 25.820939, 'lng':...",25.820939,-80.280752,NaN,FL,Adventist Bible Book Store,v-1539886822,NaN


In [58]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

In [59]:
dataframe_filtered.name

0                                  Books & Books
1                                  Booksmart Tax
2                                 Book Link Cafe
3                     Rooms2night Booking Engine
4                     Adventist Bible Book Store
5                      The First Book of Gabriel
6                                  Books & Books
7                       New Horizons - Book Room
8                             Book Barn Exchange
9                          Adventist Book Center
10                  LD Books (Libros en Español)
11               QuickBooks Support Phone Number
12                            D & B Book Binders
13                               BookLink & Cafe
14                            D & B Book Binders
15                     University Book and Video
16                           Lucianos Book Store
17                                Books 4 Travel
18                                 Booklink cafe
19                       J and R Book and Videos
20                  

In [60]:
dataframe_filtered.rename(columns={'location.address':'address','location.cc':'cc','location.city':'city','location.country':'country','location.crossStreet':'crossStreet','location.distance':'distance','location.formattedAddress': 'formattedAddress','location.labeledLatLngs':'labeledLatLngs','location.lat':'lat','location.lng':'lng','location.postalCode':'postalCode','location.state':'state'})

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Books & Books,Bookstore,265 Aragon Ave,US,Coral Gables,United States,NaN,10785,"[265 Aragon Ave, Coral Gables, FL 33134, Unite...","[{'label': 'display', 'lat': 25.75060375407932...",25.750604,-80.259944,33134,FL,4ad0d474f964a52037da20e3
1,Booksmart Tax,Financial or Legal Service,8333 NW,US,Doral,United States,"53rd Street, Suite 450",199,"[8333 NW (53rd Street, Suite 450), Doral, FL 3...","[{'label': 'display', 'lat': 25.82177731876308...",25.821777,-80.332280,33166,FL,58a5eba1bf1a6d0b9fedd1e3
2,Book Link Cafe,Coffee Shop,NaN,US,Miami,United States,NaN,6467,"[Miami, FL 33126, United States]","[{'label': 'display', 'lat': 25.79749541746666...",25.797495,-80.274939,33126,FL,4f4caf3de4b0b2a2fce7a993
3,Rooms2night Booking Engine,General Travel,8100 NW 53rd St Apt 275,US,Doral,United States,NaN,429,"[8100 NW 53rd St Apt 275, Doral, FL 33166, Uni...","[{'label': 'display', 'lat': 25.82170730116426...",25.821707,-80.329826,33166,FL,57149a2c498ece023556c1d5
4,Adventist Bible Book Store,Bookstore,NaN,US,Miami Springs,United States,NaN,5336,"[Miami Springs, FL, United States]","[{'label': 'display', 'lat': 25.820939, 'lng':...",25.820939,-80.280752,NaN,FL,517982a2e4b0545603eb185b
5,The First Book of Gabriel,Religious School,277 W 26th St,US,Hialeah,United States,NaN,5310,"[277 W 26th St, Hialeah, FL 33010, United States]","[{'label': 'display', 'lat': 25.83155040351406...",25.831550,-80.282357,33010,FL,555d4a03498ede07535e9e95
6,Books & Books,Bookstore,1640 NW 42nd Ave,US,Miami,United States,Miami International Airport,6223,[1640 NW 42nd Ave (Miami International Airport...,"[{'label': 'display', 'lat': 25.79731753851085...",25.797318,-80.277700,33126,FL,4dc7c4e4d4c0ad9c0fbebe31
7,New Horizons - Book Room,College Bookstore,NaN,US,Miami,United States,NaN,5432,"[Miami, FL 33144, United States]","[{'label': 'display', 'lat': 25.773388, 'lng':...",25.773388,-80.321575,33144,FL,4c6c0d7065eda09358cd4ad0
8,Book Barn Exchange,Bookstore,10597 SW 40th St,US,Miami,United States,NaN,10323,"[10597 SW 40th St, Miami, FL 33165, United Sta...","[{'label': 'display', 'lat': 25.73299007013759...",25.732990,-80.366857,33165,FL,4b422fdef964a5207ace25e3
9,Adventist Book Center,Bookstore,1 South Royal,US,Miami Springs,United States,Poinciana Blvd.,5321,"[1 South Royal (Poinciana Blvd.), Miami Spring...","[{'label': 'display', 'lat': 25.82105723642207...",25.821057,-80.280910,33166,FL,4ee4e024775b86c40499aff0


# Now to display the map!

In [61]:
venues_map1 = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map
# add a red circle marker to represent the Center of Downtown Miami
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Center',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map1)

# add the book related stores in blue
for lat, lng, label in zip(dataframe_filtered['location.lat'], dataframe_filtered['location.lng'], dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map1)

# display map
venues_map1

# Conclusion

# Given the following map of Toronto

In [62]:
venues_map

# and the following map of Miami

In [64]:
venues_map1

### One must conclude that there is a noticeable difference between the book related stores around Downtown Toronto and Downtown Miami. Even if one were to consider that many of the venues in Toronto use the word "book" as part of their names. And there is no correlation between that store and Acme's book accesory. Toronto would still be the better choice.

In [74]:
df.columns

Index(['categories', 'hasPerk', 'id', 'location.address', 'location.cc',
       'location.city', 'location.country', 'location.crossStreet',
       'location.distance', 'location.formattedAddress',
       'location.labeledLatLngs', 'location.lat', 'location.lng',
       'location.postalCode', 'location.state', 'name', 'referralId'],
      dtype='object')

In [75]:
dataframe.columns

Index(['categories', 'delivery.id', 'delivery.provider.icon.name',
       'delivery.provider.icon.prefix', 'delivery.provider.icon.sizes',
       'delivery.provider.name', 'delivery.url', 'hasPerk', 'id',
       'location.address', 'location.cc', 'location.city', 'location.country',
       'location.crossStreet', 'location.distance',
       'location.formattedAddress', 'location.labeledLatLngs', 'location.lat',
       'location.lng', 'location.neighborhood', 'location.postalCode',
       'location.state', 'name', 'referralId', 'venuePage.id'],
      dtype='object')